In [1]:
# tip:
#b) To register your custom env, do `from ray import tune;
#   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
#   Then in your config, do `config['env'] = [name]`.

# Import the RL algorithm (Trainer) we would like to use.
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
#import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
#import src.jsp_instance_parser 
from ray import tune
import gym

from ray import tune
from wrapper.taxi_wrapper import discretetobox

# Configure the algorithm.
def env_creator(env_config):
    env = (gym.make("Taxi-v3"))
    #env = discretetobox(gym.make("Taxi-v3"))

    #env = gym.make('LunarLander-v2')
    return env

# use tune to register the custom environment for the ppo trainer
tune.register_env('TaxiTaxi',env_creator)


config = {
    # Environment (RLlib understands openAI gym registered strings).
    #"env": "CartPole-v1",
    "env": "Taxi-v3",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 1,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    #"horizon":1,
    "evaluation_duration":10,
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    },
    
}

# function which returns environment, by now config is unused...


# use tune to register the custom environment for the ppo trainer
#tune.register_env('CartPole-v1',env_creator)
#tune.register_env('Taxi-v3',env_creator)
# Create our RLlib Trainer.
trainer = PPOTrainer(config=config)

#trainer = AlphaZeroTrainer(config=config)




2022-08-24 20:11:44,821	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
2022-08-24 20:11:55,718	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-08-24 20:11:55,719	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
2022-08-24 20:11:55,781	INFO trainable.py:159 -- Trainable.setup took 10.966 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from ray import tune
#path='resources/jsp_instances/standard/abz8.txt'

#env = gym.make('CartPole-v1')
#env = gym.make('Taxi-v3')

#env = gym.make('LunarL')
#env.render()
# run until episode ends
# list that lists all actions
iteration_list=[]
epoch_nr=0
for _ in range(300):
    epoch_nr+=1
    trainer.train()
    print(f"trainer trained: {epoch_nr} Epochs")
    #trainer.save(f"checkpoints/rllib_checkpoint{_+1}")
    



2022-08-24 20:10:49,561	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


trainer trained: 1 Epochs
trainer trained: 2 Epochs
trainer trained: 3 Epochs
trainer trained: 4 Epochs
trainer trained: 5 Epochs
trainer trained: 6 Epochs
trainer trained: 7 Epochs
trainer trained: 8 Epochs
trainer trained: 9 Epochs


In [ ]:
# use string number to restore
#nr_restore="50"
#trainer.restore(f'checkpoints/rllib_checkpoint{nr_restore}/checkpoint_{nr_restore.zfill(6)}/checkpoint-{nr_restore}')


In [13]:
from ray.rllib.contrib.alpha_zero.environments.cartpole import CartPole
#env = gym.make('CartPole-v1')
#env = discretetobox(gym.make("Taxi-v3"))
episode_reward = 0
env = (gym.make("Taxi-v3"))

done = False
obs = env.reset()
iterations=0
while not done:
    action = trainer.compute_action(obs)
    #print(f"action: {action}")
    obs, reward, done, info = env.step(action)
    episode_reward += reward
    #env.render(mode="human")
    iterations += 1
env.render()
env.close()
print(episode_reward)
print(iterations)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
-290
200


In [2]:
SELECT_ENV = "Taxi-v3"
N_ITER = 10
config["log_level"] = "WARN"

agent = trainer = PPOTrainer(config=config)
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. ')

2022-08-24 20:11:55,826	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
2022-08-24 20:12:06,487	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-08-24 20:12:06,489	WARNING trainer.py:2540 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
2022-08-24 20:12:06,594	INFO trainable.py:159 -- Trainable.setup took 10.771 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-08-2

  1: Min/Mean/Max reward: -875.0000/-755.2500/-457.0000, len mean: 193.0500. 
  2: Min/Mean/Max reward: -929.0000/-766.2500/-457.0000, len mean: 196.1000. 
  3: Min/Mean/Max reward: -929.0000/-735.4098/-256.0000, len mean: 195.3115. 
  4: Min/Mean/Max reward: -929.0000/-697.3415/-69.0000, len mean: 194.5122. 
  5: Min/Mean/Max reward: -929.0000/-653.6300/-69.0000, len mean: 195.5000. 
  6: Min/Mean/Max reward: -929.0000/-586.5000/-69.0000, len mean: 196.8900. 
  7: Min/Mean/Max reward: -776.0000/-504.7600/-69.0000, len mean: 195.9700. 
  8: Min/Mean/Max reward: -695.0000/-434.7200/-69.0000, len mean: 197.2700. 
  9: Min/Mean/Max reward: -569.0000/-375.8200/-151.0000, len mean: 198.3100. 
 10: Min/Mean/Max reward: -497.0000/-334.3300/-151.0000, len mean: 198.3100. 


In [ ]:
import argparse

import ray
from ray.tune.registry import register_env
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from ray.rllib.contrib.alpha_zero.environments.cartpole import CartPole
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.models.catalog import ModelCatalog


#parser = argparse.ArgumentParser()
#parser.add_argument("--training-iteration", default=2, type=int)
#args = parser.parse_args()
ray.shutdown()
ray.init()

ModelCatalog.register_custom_model("dense_model", DenseModel)
register_env("CartPoleEnv", lambda _: CartPole())

config = {
    "num_workers"       : 0,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 1e-4,
    "num_sgd_iter"      : 1,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.0,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : False,
        "add_dirichlet_noise": True,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",
    },
}

agent = AlphaZeroTrainer(config=config, env="CartPoleEnv")

#for _ in range(args.training_iteration):
for _ in range(1):
    agent.train()

env = CartPole()
episode_reward = 0
done = False


In [ ]:
obs = env.reset()
while not done:
    print(obs)
    action = agent.compute_single_action(obs)
    obs, episode_reward, done, info = env.step(action)

print(episode_reward)

ray.shutdown()

In [ ]:
from ray import rllib, tune
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.policy.policy_map import PolicyMap
from ray.rllib.evaluation.episode import MultiAgentEpisode



#agent = AlphaZeroTrainer(env=CartPole, config=config)
agent = AlphaZeroTrainer(env=CartPole, config=config)
for _ in range(1):
    agent.train()

policy = agent.get_policy(DEFAULT_POLICY_ID)

env = CartPole()
obs = env.reset()

episode = MultiAgentEpisode(
    PolicyMap(0,0),
    lambda _, __: DEFAULT_POLICY_ID,
    lambda: None,
    lambda _: None,
    0,
)

episode.user_data['initial_state'] = env.get_state()

done = False

while not done:
    action, _, _ = policy.compute_single_action(obs, episode=episode)
    print(action)
    obs, reward, done, _ = env.step(action)
    print(obs, reward)
    episode.length += 1

assert reward == episode.length

In [ ]:

policy = agent.get_policy(DEFAULT_POLICY_ID)

env = CartPole()
obs = env.reset()

episode = MultiAgentEpisode(
    PolicyMap(0,0),
    lambda _, __: DEFAULT_POLICY_ID,
    lambda: None,
    lambda _: None,
    0,
)

episode.user_data['initial_state'] = env.get_state()

done = False

while not done:
    action, _, _ = policy.compute_single_action(obs, episode=episode)
    print(action)
    obs, reward, done, _ = env.step(action)
    env.render()
    episode.length += 1

assert reward == episode.length